GOOFSPIEL
Gdy zagramy kartę wyższą niż przeciwnik dostajemy punkty równe wartości karty na stole, gdy zagramy to samo, nikt nie dostaje punktów.

Korzystając z oznaczeń poleconeg artykułu oznaczmy f(V,Y,P) jako zwrot gracza gdzie V=nasze karty, Y=karty przeciwnika, P=karty w talii.


Niech f_k(V,Y,P) oznacza tą wartość dla odkrytej karty o wartości k. Zatem można z tego wywnioskować że nasza oczekiwana wartość gry jest średnią wszystkich f_k

X_i,j=P_k*sign(V_i-Y_j)+f(V\V_i,Y\Y_j,P\P_k)  czyli w skrócie +-wartość karty odkrytej w zależności czy wygraliśmy, lub 0 w remisie powiększone o oczekiwaną wartość reszty gry


Trudność może wystąpić przy wyliczaniu wartości oczekiwanej reszty gry dlatego podejdziemy do tego rekurencyjnie

In [13]:
k=4                                                                     #Podajemy ilość elementów w listach
V=[*range(1, k+1, 1)]                                                   #tworzymy listy
Y=[*range(1, k+1, 1)]
P=[*range(1, k+1, 1)]
wynik=matrix(RDF,k,k,0)

In [2]:
def sign(x):                                                            #zdefiniujmy funkcję sign(x)
    if x==0:
        return 0
    else:
        return math.copysign(1,x)

In [3]:
def f(V,Y,P,N):                                                         #funkcja f, czyli średnia z fk czyli wszystkich możliwych pozostałych gier
    if N==0:
        return 0
    elif N==1:                                                          #N==1 podajemy ręcznie aby nie robić niepotrzebnych obliczeń
        return sign(V[0]-Y[0])*P[0]
    else:                                                               #dla n>1 wywołujemy funkcję fk czyli naszą rekurencje
        avg=0
        for Pk in range(N):
            avg+=fk(V,Y,P,Pk,N)
        return avg/N

In [4]:
def fk(V,Y,P,Pk,N):                                                     #funkcja fk czyli pozostałe gry po odkryciu karty Pk
    if N==0:
        return 0
    else:
        X=matrix(RR,N,N,0)                                              #tworzymy pustą macierz na której będziemy nadpisywać tworząc macierz wypłat
        P2=P.copy()
        P2.remove(P[Pk])
        for i in range(N):
            V2=V.copy()
            V2.remove(V[i])
            for j in range(N):                                          #tworzymy kopie list aby je modyfikować na potrzeby dalszego kroku rekurencji
                Y2=Y.copy()
                Y2.remove(Y[j])
                X[i,j]=P[Pk]*sign(V[i]-Y[j])+f(V2,Y2,P2,N-1)            #przypisujemy macierzy X wartości ze wzoru, testowane na N=2 podaje poprawną macierz
        #if N==k+2:
        #show(X)
        lp=MixedIntegerLinearProgram(maximization=True)                 #szukanie optymalnej strategii gry z macierzą wypłat X
        x=lp.new_variable(nonnegative=True)
        y=lp.new_variable(nonnegative=False)
        lp.set_objective(y[0])                                          #cel y[0] czyli dodatkowa zmienna
        lp.add_constraint(sum([x[c] for c in range(N)])==1)             #suma prawdopodobieństw z którą mamy grać dane strategie musi być równa 1
        for a in range(N):
            lp.add_constraint(sum([round(X[b,a],10)*x[b] for b in range(N)])-y[0]>=0)           #dodajemy warunki z liczenia strategii optymalnej
        lp.solve()
        if N==k:
            for h in range(N):
                wynik[h,Pk]=round(lp.get_values(x[h]),4)
            #show(lp.get_values(x))                                      #pokazuje wartości prawdopodobieństwa zagrań danej karty przy odkryciu konkretnej karty (trzeba transponować)
            if Pk==k-1:
                show(wynik)
    return lp.get_objective_value()                                     #funkcja zwraca wartość funkcji celu


In [6]:
print("Macierz optymalnej strategii dla N=2:")
f(V,Y,P,k)

Macierz optymalnej strategii dla N=2:


[1.0 0.0]
[0.0 1.0]

0.0

In [8]:
print("Macierz optymalnej strategii dla N=3:")
f(V,Y,P,k)

Macierz optymalnej strategii dla N=3:


[   1.0    0.0    0.0]
[   0.0 0.5556    0.0]
[   0.0 0.4444    1.0]

0.0

In [10]:
print("Macierz optymalnej strategii dla N=4:")
f(V,Y,P,k)

Macierz optymalnej strategii dla N=4:


[   1.0 0.3371 0.2687    0.0]
[   0.0  0.136    0.0    0.0]
[   0.0 0.5269  0.514    0.0]
[   0.0    0.0 0.2173    1.0]

1.474514954580286e-17

In [12]:
print("Macierz optymalnej strategii dla N=5:")
f(V,Y,P,k)

Macierz optymalnej strategii dla N=5:


[ 0.047 0.1855 0.1182 0.1226 0.1123]
[0.8327    0.0 0.1188 0.0735 0.0241]
[0.1203 0.7375    0.0 0.1915    0.0]
[   0.0  0.077  0.763 0.2043    0.0]
[   0.0    0.0    0.0 0.4081 0.8636]

6.38378239159465e-17